# 🪨 Strike Weather: A Data Journey into Summer 1984

## 🏛️ A Visit to the National Museum in Cardiff

Recently, I visited the National Museum in Cardiff. Among the many exhibitions, one in particular caught my attention — a display on the miners’ strike in the 1980s. 

One small detail lingered in my mind: a sign noting how miners remembered the summer of 1984 for its heat. They called it **"strike weather."** No temperature was listed, no graphs shown. But the memory was vivid — and shared.

I became curious: **was that summer really unusually hot?** Could I use data to understand their experience better — not to question it, but to explore what might have made it memorable?


In [1]:
import pandas as pd
import requests
import matplotlib.pyplot as plt

def fetch_and_clean_met_data(station_name, url):
    """
    Fetch and clean Met Office station data.

    Parameters:
    - station_name (str): A short name for the station (used in the DataFrame)
    - url (str): URL to the Met Office .txt data file

    Returns:
    - pd.DataFrame: Cleaned DataFrame with columns:
      [Year, Month, Tmax, Tmin, AF, Rain, Sun, Station]
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"[{station_name}] Error fetching data: {e}")
        return None

    text_lines = response.text.split('\n')
    
    # Find where the data starts
    for i, line in enumerate(text_lines):
        if line.strip().startswith("yyyy"):
            data_start_line = i + 1
            break

    # Extract data lines
    data_rows = []
    for line in text_lines[data_start_line:]:
        parts = line.split()
        if len(parts) == 7:
            data_rows.append(parts)
    
    # Convert to DataFrame
    columns = ["Year", "Month", "Tmax", "Tmin", "AF", "Rain", "Sun"]
    df = pd.DataFrame(data_rows, columns=columns)

    # Replace missing values
    df.replace(['---', ''], pd.NA, inplace=True)
    
    # Clean numeric columns
    numeric_columns = ["Tmax", "Tmin", "AF", "Rain", "Sun"]
    for col in numeric_columns:
        df[col] = df[col].str.replace('*', '', regex=False)
        df[col] = pd.to_numeric(df[col], errors='coerce')
    
    # Clean Year and Month
    df.dropna(subset=["Year", "Month"], inplace=True)
    df["Year"] = df["Year"].astype(int)
    df["Month"] = df["Month"].astype(int)
    
    df["Station"] = station_name
    return df

# Example: URLs for three stations
station_urls = {
    "Sheffield": "https://www.metoffice.gov.uk/pub/data/weather/uk/climate/stationdata/sheffielddata.txt",
    "Waddington": "https://www.metoffice.gov.uk/pub/data/weather/uk/climate/stationdata/waddingtondata.txt",
    "Cardiff": "https://www.metoffice.gov.uk/pub/data/weather/uk/climate/stationdata/cardiffdata.txt"
}

# Fetch all data into one DataFrame
all_data = pd.concat(
    [fetch_and_clean_met_data(name, url) for name, url in station_urls.items()],
    ignore_index=True
)

all_data 

# Define the URL 
# sheffield_url = "https://www.metoffice.gov.uk/pub/data/weather/uk/climate/stationdata/sheffielddata.txt"
# waddington_url = "https://www.metoffice.gov.uk/pub/data/weather/uk/climate/stationdata/waddingtondata.txt"
# cardiff_url = "https://www.metoffice.gov.uk/pub/data/weather/uk/climate/stationdata/cardiffdata.txt"

# # Use the function for a single station
# sheffield_df = fetch_and_clean_met_data("Sheffield", sheffield_url)
# waddington_df = fetch_and_clean_met_data("Waddington", waddington_url)
# cardiff_df = fetch_and_clean_met_data("Cardiff", cardiff_url)
# # # Concatenate the dataframes
# # all_data = pd.concat([sheffield_df, waddington_df, cardiff_df], ignore_index=True)
# # # Save to CSV
# # all_data.to_csv("met_office_data.csv", index=False)

# # Preview the data
# sheffield_df

,Year,Month,Tmax,Tmin,AF,Rain,Sun,Station
0,1883,1,6.3,1.7,6.0,122.1,NaN,Sheffield
1,1883,2,8.0,2.8,2.0,69.8,NaN,Sheffield
2,1883,3,4.8,-1.6,23.0,29.6,NaN,Sheffield
3,1883,4,12.2,3.8,2.0,74.0,NaN,Sheffield
4,1883,5,14.7,6.2,0.0,31.2,NaN,Sheffield
...,...,...,...,...,...,...,...,...
3167,2023,8,21.4,12.8,0.0,94.2,NaN,Cardiff
3168,2023,9,22.0,13.1,0.0,107.0,NaN,Cardiff
3169,2023,10,17.4,9.6,0.0,178.4,NaN,Cardiff
3170,2023,11,12.2,5.1,4.0,202.0,NaN,Cardiff


In [ ]:
import plotly.express as px
import plotly.graph_objects as go

# <iframe src="/Users/nat/Desktop/miner_strike_heatwave/code/output/met_office_table.html" width="100%" height="600"></iframe>

# Display interactive table using Plotly DataTable
fig = go.Figure(data=[
    go.Table(
        header=dict(values=list(all_data.columns), fill_color='paleturquoise', align='left'),
        cells=dict(values=[all_data[col] for col in all_data.columns], fill_color='lavender', align='left')
    )
])

fig.update_layout(title="Interactive Weather Data For 3 Stations", title_x=0.5)
fig.show()

# Save as HTML
fig.write_html("/Users/nat/Desktop/miner_strike_heatwave/code/output/met_office_table.html")


In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Assuming 'all_data' DataFrame is already created (as per your previous code)

fig = go.Figure(go.Table(
    header={"values": all_data.columns.tolist()},
    cells={"values": all_data.T.values.tolist()}
))

# Add filtering buttons for Station and Year
fig.update_layout(
    updatemenus=[
        {
            "buttons": [{"label": "Stations", "method": "update", "args": [{"cells": {"values": all_data.T.values.tolist()}}]}]
            + [{"label": station, "method": "update", "args": [{"cells": {"values": all_data.loc[all_data["Station"] == station].T.values.tolist()}}]} for station in all_data["Station"].unique()],
            "direction": "down", "showactive": True, "x": -0.2, "xanchor": "left", "y": 1.15, "yanchor": "top"
        },
        {
            "buttons": [{"label": "All Years", "method": "update", "args": [{"cells": {"values": all_data.T.values.tolist()}}]}]
            + [{"label": str(year), "method": "update", "args": [{"cells": {"values": all_data.loc[all_data["Year"] == year].T.values.tolist()}}]} for year in sorted(all_data["Year"].unique())],
            "direction": "down", "showactive": True, "x": -0.2, "xanchor": "left", "y": 1.0, "yanchor": "top"
        }
    ]
)

# Add title to the plot
fig.update_layout(title="    Met Office Weather Data by Station and Year")

# Show the table
fig.show()

fig.write_html("/Users/nat/Desktop/miner_strike_heatwave/code/output/met_office_table.html")


## 🌡️ Was 1984 a Hot Summer?

Using data from the UK Met Office for Cardiff, I looked at **monthly maximum temperatures (Tmax)** for 1984 and compared them to other years.

Here's what June - August look like in 1984:


In [ ]:
import pandas as pd  
import plotly.express as px 

# --- Step 1: Filter the summer months ---
# Here, we filter the data to include only the summer months (June, July, August).
# The 'Month' column in the dataset stores the month of each record, and we use `.isin()` to select the relevant months.
summer_data = all_data[all_data['Month'].isin([6, 7, 8])]  # June = 6, July = 7, August = 8

# --- Step 2: Plotting the Maximum Summer Temperature (Tmax) ---
# We will create a line plot to visualize how the maximum temperature (Tmax) changed over time for each station.

fig_temp = px.line(
    summer_data,  # The data to plot, filtered for summer months
    x="Year",  # The x-axis will represent the 'Year'
    y="Tmax",  # The y-axis will represent the 'Tmax' (maximum temperature) column
    color="Station",  # This will differentiate the data points by station, so each station gets its own color
    title="📈 Max Summer Temperature (Tmax) by Station and Year",  # The title of the plot
    labels={"Tmax": "Max Temperature (°C)", "Year": "Year"},  # Customizing the labels for axes
    markers=True  # Adds a marker (a dot) for each data point to make it more visually clear
)

# --- Step 3: Customize the Layout ---
# Here, we further customize the appearance of the plot by adjusting titles and template.
fig_temp.update_layout(
    xaxis_title="Year",  # Label for the x-axis
    yaxis_title="Mean Tmax (°C)",  # Label for the y-axis
    template="plotly_white"  # This applies a white background theme for the plot
)

# --- Step 4: Show the Plot ---
fig_temp.show()  # This will display the plot in the notebook or browser

# --- Step 5: Save the Plot as an HTML File ---
# After creating the plot, you can save it as an interactive HTML file.
# The saved file can be opened in any web browser.
fig_temp.write_html("/Users/nat/Desktop/miner_strike_heatwave/code/output/max_summer_temperature_by_station.html")

# Optionally, you can also save the plot as a static image (e.g., PNG) using the line below:
# fig_temp.write_image("max_summer_temperature_by_station.png")


## ☀️🌧️ Sunshine and Rainfall — Feeling the Heat Differently

To explore this further, I also looked at **rainfall** and **sunshine hours** for the summer of 1984. Fewer rainy days and more sunshine might suggest a drier, more oppressive kind of heat — the kind that sticks. 

Here's what the data says:


In [ ]:
# --- Plot 2: Total Rainfall over Time ---
# We will create a line plot to visualize how total rainfall changed over time for each station.

fig_rain = px.line(
    summer_data,  # The data to plot, filtered for summer months
    x="Year",  # The x-axis will represent the 'Year'
    y="Rain",  # The y-axis will represent the 'Rain' (total rainfall) column
    color="Station",  # This will differentiate the data points by station, so each station gets its own color
    title="🌧️ Total Summer Rainfall by Station and Year",  # The title of the plot
    labels={"Rain": "Total Rainfall (mm)", "Year": "Year"},  # Customizing the labels for axes
    markers=True  # Adds a marker (a dot) for each data point to make it more visually clear
)

# --- Customize the Layout ---
# Here we further customize the appearance of the plot by adjusting titles and theme.
fig_rain.update_layout(
    xaxis_title="Year",  # Label for the x-axis
    yaxis_title="Total Rainfall (mm)",  # Label for the y-axis
    template="plotly_white"  # This applies a white background theme for the plot
)

# --- Show the Plot ---
fig_rain.show()  # This will display the interactive plot in the notebook or browser

# --- Save the Plot as an HTML File ---
# Save the rainfall plot as an interactive HTML file.
fig_rain.write_html("/Users/nat/Desktop/miner_strike_heatwave/code/output/total_summer_rainfall_by_station.html")

# --- Plot 3: Total Sunshine Hours over Time ---
# Now, we will create a line plot to visualize how total sunshine hours changed over time for each station.

fig_sun = px.line(
    summer_data,  # The data to plot, filtered for summer months
    x="Year",  # The x-axis will represent the 'Year'
    y="Sun",  # The y-axis will represent the 'Sun' (total sunshine hours) column
    color="Station",  # This will differentiate the data points by station, so each station gets its own color
    title="☀️ Total Summer Sunshine Hours by Station and Year",  # The title of the plot
    labels={"Sun": "Total Sunshine (hrs)", "Year": "Year"},  # Customizing the labels for axes
    markers=True  # Adds a marker (a dot) for each data point to make it more visually clear
)

# --- Customize the Layout ---
# Here we further customize the appearance of the plot by adjusting titles and theme.
fig_sun.update_layout(
    xaxis_title="Year",  # Label for the x-axis
    yaxis_title="Total Sunshine (hrs)",  # Label for the y-axis
    template="plotly_white"  # This applies a white background theme for the plot
)

# --- Show the Plot ---
fig_sun.show()  # This will display the interactive plot in the notebook or browser

# --- Save the Plot as an HTML File ---
# Save the sunshine plot as an interactive HTML file.
fig_sun.write_html("/Users/nat/Desktop/miner_strike_heatwave/code/output/total_summer_sunshine_by_station.html")


## 🌞 How Unusual Was Summer 1984? 🔥

This code analyzes summer weather data from multiple UK weather stations. It focuses on the summer months (June, July, August) and calculates **average** and **total** values for:

- 🌡️ Temperature
- 🌧️ Rainfall
- ☀️ Sunshine

The data is grouped by **station** and **year**, and **z-scores** are calculated to compare each year's values against a climatology baseline (1977–2006). 

The **z-scores** help identify how unusual a year is compared to the baseline. 📊 The code then generates **interactive scatter plots** to visualize the relationship between temperature and rainfall, with a special focus on the summer of 1984. 🗓️


In [ ]:
import plotly.express as px

# Write explanatory paragraph about the code below and explain the methodology


# --- STEP 1: FILTER FOR SUMMER MONTHS ONLY ---
# We only care about summer (June, July, August = months 6, 7, 8)
df_summer = all_data[all_data["Month"].isin([6, 7, 8])]

# --- STEP 2: GROUP BY STATION AND YEAR ---
# We want to summarize data for each year, at each weather station.
# We're calculating averages and totals for temperature, rainfall, and sunshine.
# .agg() allows us to compute multiple summary stats at once
summer_by_year = df_summer.groupby(["Station", "Year"]).agg(
    mean_temp=("Tmax", "mean"),       # Average of daily max temperatures
    max_temp=("Tmax", "max"),         # Hottest single day in summer
    mean_sun=("Sun", "mean"),         # Average daily sunshine hours
    total_sun=("Sun", "sum"),         # Total sunshine for the summer
    mean_rain=("Rain", "mean"),       # Average daily rainfall
    total_rain=("Rain", "sum")        # Total summer rainfall
).reset_index()

# Round numbers to 2 decimal places for readability
summer_by_year = summer_by_year.round(2)

# --- STEP 3: LOOP OVER EACH WEATHER STATION ---
# This makes sure each station gets its own customized plot
stations = summer_by_year["Station"].unique()

for station in stations:
    # Work with just one station's data at a time
    station_data = summer_by_year[summer_by_year["Station"] == station].copy()

    # --- STEP 4: DEFINE CLIMATOLOGY BASELINE (1977–2006) ---
    # This is the "normal" period we'll compare everything to
    # See WMO climate baseline periods: https://public.wmo.int/en/our-mandate/climate/wmo-climate-normals
    clim_base = station_data[(station_data["Year"] >= 1977) & (station_data["Year"] <= 2006)]

    # Calculate the MEAN and STANDARD DEVIATION for all numeric values
    # These will help us calculate z-scores (standardized differences)
    clim_mean = clim_base.mean(numeric_only=True)
    clim_std = clim_base.std(numeric_only=True)

    # --- STEP 5: CALCULATE Z-SCORES FOR TEMPERATURE AND RAINFALL ---
    # A z-score tells us how unusual a year is, compared to the baseline
    # e.g., z = (value - average) / std deviation
    # Learn about z-scores: https://statisticsbyjim.com/basics/z-scores/
    station_data["z_temp"] = (station_data["mean_temp"] - clim_mean["mean_temp"]) / clim_std["mean_temp"]
    station_data["z_rain"] = (station_data["total_rain"] - clim_mean["total_rain"]) / clim_std["total_rain"]

    # --- STEP 6: ADD LABELS FOR PLOTTING ---
    # We'll use these for coloring and tooltips in the plot
    station_data["label"] = station_data["Year"].astype(str)  # Convert year to string for labels
    station_data["color"] = "Climatology (1977–2006)"         # Default group label
    station_data.loc[station_data["Year"] > 2006, "color"] = "After 2006"  # Years after baseline
    station_data.loc[station_data["Year"] == 1984, "color"] = "1984"       # Highlight the 1984 strike year

    # --- STEP 7: MAKE THE PLOTLY SCATTER PLOT ---
    # Plotly Express makes interactive plots easily: https://plotly.com/python/plotly-express/
    fig = px.scatter(
        station_data,
        x="z_temp",                   # X-axis: How hot was the summer?
        y="z_rain",                   # Y-axis: How rainy was the summer?
        color="color",               # Color points by their category
        text="label",                # Show the year as a label
        hover_data={                 # Customize what's shown on hover
            "Year": True,
            "mean_temp": True,
            "total_rain": True,
            "z_temp": ":.2f",        # Format: 2 decimal places
            "z_rain": ":.2f",
            "color": False,
            "label": False
        },
        labels={                     # Axis labels
            "z_temp": "Z-score: Mean Summer Temperature",
            "z_rain": "Z-score: Total Summer Rainfall"
        },
        title=f"☀️ {station} Summer 1984 Compared to Climatology (1977–2006)",
        color_discrete_map={         # Manual color mapping
            "Climatology (1977–2006)": "#1f77b4",  # Blue
            "1984": "#d62728",                    # Red
            "After 2006": "#7f7f7f"               # Gray
        }
    )

    # --- STEP 8: STYLE THE POINT FOR 1984 (make it stand out) ---
    fig.update_traces(
        marker=dict(size=10),             # General point size
        textposition='top center'         # Place year label above point
    )
    fig.update_traces(
        marker=dict(size=14, symbol="diamond"),   # Bigger, diamond shape for 1984
        selector=dict(name="1984")
    )

    # --- STEP 9: ADD CROSSHAIR LINES AT 0 ---
    # Helps the viewer interpret where "average" is (z = 0)
    fig.add_vline(x=0, line_dash="dash", line_color="gray")
    fig.add_hline(y=0, line_dash="dash", line_color="gray")

    # --- STEP 10: ANNOTATE THE 1984 POINT WITH ITS Z-SCORES ---
    # if not station_data[station_data["Year"] == 1984].empty:
    #     z_temp_1984 = station_data.loc[station_data["Year"] == 1984, "z_temp"].values[0]
    #     z_rain_1984 = station_data.loc[station_data["Year"] == 1984, "z_rain"].values[0]

    #     fig.add_annotation(
    #         x=z_temp_1984, y=z_rain_1984,
    #         text=f"1984\nTemp Z={z_temp_1984:.2f}\nRain Z={z_rain_1984:.2f}",
    #         showarrow=True,
    #         arrowhead=2,
    #         ax=30,
    #         ay=-40,
    #         bgcolor="rgba(255,255,255,0.7)",  # Light background behind text
    #         font=dict(size=12)
    #     )

    # --- STEP 11: FINAL POLISHING ---
    fig.update_layout(
        legend_title="",
        template="plotly_white",    # Clean white background
        xaxis=dict(zeroline=True, zerolinecolor='black'),
        yaxis=dict(zeroline=True, zerolinecolor='black'),
        legend_traceorder="normal"
    )

    # --- STEP 12: SHOW THE PLOT ---
    # This will open the interactive chart in Jupyter, Colab, or browser
    fig.show()
    # --- STEP 13: SAVE THE PLOT ---
    fig.write_html(f"/Users/nat/Desktop/miner_strike_heatwave/code/output/{station}_summer_1984_vs_climatology.html")
# --- END OF SCRIPT ---

### Method & Interpretation

#### 🧭 **Why Use 1977–2006 as a Baseline?**
We use the period from 1977 to 2006 as our climate reference for the following reasons:

- ✅ **Standard Climate Period**: This 30-year block is the standard used in climate science to define "normal" conditions.
- 🌍 **Pre-Global Warming Surge**: This period predates the more recent rapid global warming, providing a stable baseline for comparing past climate conditions.
- 📊 **Reliable Base for Comparison**: By using this timeframe, we can measure how unusual or extreme subsequent summers have been compared to a stable climate period.

---

#### 🔍 **What This Plot Shows**
This scatter plot helps us understand how unusual each Cardiff summer (June–August) was, from 1977 onward, based on temperature and rainfall.

- **Each dot** represents one summer.
- **X-axis (z_temp)**: Shows how much hotter or cooler the summer was compared to the 1977–2006 baseline (i.e., the temperature anomaly).
- **Y-axis (z_rain)**: Shows how much wetter or drier the summer was compared to the same baseline (i.e., the rainfall anomaly).

---

#### 🧪 **How to Read the Z-scores:**
- **Z ≈ 0** → The summer was close to the baseline (normal conditions).
- **Z ≈ +1 to +2** → Noticeably hotter or drier than normal.
- **Z ≥ +2** → Extremely hot or dry conditions (e.g., heatwaves or droughts).
- **Z ≤ -1** → Cooler or wetter than the baseline (e.g., rainy or cold summers).

Each summer is plotted in one of four quadrants based on its z-scores:

- **Top-right** → Hot & Wet 🌴
- **Top-left** → Cool & Wet 🌧️
- **Bottom-left** → Cool & Dry 🌬️
- **Bottom-right** → Hot & Dry 🌞 ← This is where **1984** falls.

---

#### 📊 **Quadrant Breakdown**

| Quadrant             | Temp Z-Score (x-axis) | Rain Z-Score (y-axis) | What It Means                    | Example Interpretation         |
|----------------------|-----------------------|------------------------|----------------------------------|--------------------------------|
| **Top-right**        | Positive (hot)        | Positive (wet)         | Hotter **and** wetter than usual | Tropical-feeling summer, storms maybe |
| **Top-left**         | Negative (cool)       | Positive (wet)         | Cooler **and** wetter than usual | Damp, grey, maybe a washout    |
| **Bottom-left**      | Negative (cool)       | Negative (dry)         | Cooler **and** drier than usual  | Cold, dry — maybe boring       |
| **Bottom-right**     | Positive (hot)        | Negative (dry)         | Hotter **and** drier than usual  | Heatwave or drought conditions |

---

#### 🧐 **Interpreting Summer 1984** 
- **1984** falls in the **Bottom-right quadrant**: 
- **Hot**: The temperature was significantly higher than normal.
- **Dry**: The rainfall was much lower than typical summer conditions.
- This combination represents **a heatwave or drought**, making 1984 an especially unusual summer in Cardiff, marked by high temperatures and low rainfall.



## 🎤 Conclusion

This small exploration is not meant to *quantify* a protest — it can’t.  
But it helps highlight how **climate and weather** became part of the **collective memory** of striking miners. “Strike weather” wasn’t just about temperature; it was about time, tension, and lived experience.

So was it truly ‘strike weather’? According to the data, 1984 was indeed warmer and sunnier than average, especially in June and July. But memory doesn’t just store numbers. The heat, the tension, the struggle — it all blurs into that dry, vivid summer. And that’s the beauty of data: it doesn’t dismiss memory. It helps us understand it.